<a href="https://colab.research.google.com/github/alirezakavianifar/machineLearning/blob/main/ANN_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import necessary libraries and load the dataset


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
url="https://github.com/2blam/ML/raw/master/deep_learning/Churn_Modelling.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

## Split the data into independant and dependant vatiables and convert categorical variables

In [12]:
X = df.iloc[:, 3:13]
y = df.iloc[:, 13]

geography = pd.get_dummies(X["Geography"],drop_first=True)
gender = pd.get_dummies(X["Gender"], drop_first=True)

In [14]:
X = pd.concat([X,geography,gender], axis=1)

In [15]:
X = X.drop(['Geography','Gender'], axis=1)

## Split the data into training and testing

from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=5)

# Scale the data

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Importing keras libraries

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense